This notebook contains all my notes from Udacity Course [Artificial Intelligence for Robotics](https://www.udacity.com/course/artificial-intelligence-for-robotics--cs373).  I have learned a lots of fundamental concept of autonomous driving from this course and I am thankful to Udacity for that.

## Localization

Localization is the process, how a robot identify its location in an environment. A robot does this using probability theory. Initially the robot does not know where it is, so the probability of being in any place will be the same. In localization, basically we try to change these probabilities with the measurement of the robot such as the probability of one place increases whereas probability of other places decreases. The location with highest probability is the location of the robot and thus our robot will localize itself in the environment. 

##### steps in localization are:

   1. Initialize all the locations with uniform probability distribution, which is called prior belief.
   2. sense the world and change our prior belief according to the sense, which is called posterior belief. To change our prior we have to multiply it with the factor of the sense being correct or wrong. Further we have to normalize this to make it a valid probability distribution.   
   3. Take an action and change the posterior belief according to the action taken, which is called convolution. 
   4. sense the world again and now changing our prior belief from previous step will do the trick, now our posterior belief will be something meaningfull. Only one location will have the highest probability. So thus our robot will localize itself. 

To understand it, lets assume a robot in one dimensional discrete world. We can define the location of this world with different colors and our robot can sense these colors. Now we will try to apply the above algorithm on this robot and lets whether our robot can localize itself or not:

In [9]:
# one dimensional world
world = ['red', 'green', 'green','red', 'red', 'green']
# step 1: Initializing all the locations with uniform probability
p = [1/len(world) for _ in range(len(world))]
# These are the factor of being the sense correct or not 
pHit = 0.6 
pMiss = 0.2

# These are the valid action the robot can take
action = [0,1] #0 means move left and 1 means move right

# we will do step 2 as a function because we have to do it repeatedly.
def sense(sense, prior):
    '''This function can calculate posterior probability given the sense and prior'''
    posterior = [0.0 for _ in range(len(prior))]
#     chaning prior according to the factor
    for i in range(len(prior)):
        if sense==world[i]:
            posterior[i] = prior[i] * pHit
        else:
            posterior[i] = prior[i] * pMiss
#     normalizing the posterior to make it a valid probability distribution
    posterior = [p/sum(posterior) for p in posterior]
    return posterior


# we will do step 3 as a function too, cause we have to do it repeatedly too. 
def convolution(action, posterior):
    '''This function will change the posterior according to the action taken by the robot.
    This function assumes the world to be a cyclic world that means the element falls off 
    from right will go to the left or vice versa'''
    new_prior = [0.0 for _ in range(len(posterior))]
#   robot moves to the left
    if action == 0:
        new_prior = [posterior[(i+1)%len(posterior)] for i in range(len(posterior))]
#     robot moves to the right
    else:
        new_prior = [posterior[(i-1)] for i in range(len(posterior))]
    return new_prior

# applying the algorithm
# step 2
posterior = sense('green',p)
# step 3
new_prior = convolution(1, posterior)
# step 2 repeated
new_posterior = sense('green',new_prior)
print(new_posterior)

[0.13636363636363638, 0.13636363636363638, 0.4090909090909091, 0.13636363636363638, 0.04545454545454547, 0.13636363636363638]


As we can see the highest probability is in location 2(starting from 0 location) which is correct according to our world setup, if we sense two times and green is the result for each time. So by applying the above algorithm our robot can actually localize itself in this one dimensional world.

### Inexact robot motion

So we have undersatnd the basic concepts of localization. But in our previous example we assume the robot to be in exact motion. Which means if the robot wants to go to right by one cell it does it correctly every time. This is not the case in real life. In reality, the robot motion is uncertain. 

So for example, lets say if we give the robot the command to move right by one cell, the robot remains in the current cell with probability of 0.1, it moves to the right cell with probability of 0.8, it can also overshoot the goal by one cell with probability of 0.1. 

Considering this uncertain robot motion, the new probability of a cell will be the addition of all the probabilities from cells those are candidate for the current cell.  

In [17]:
world=['green', 'red', 'red', 'green', 'green']
p = [1/len(world) for _ in range(len(world))]
measurements = ['red', 'green'] # considering multiple measrements
motions = [1, 1] # considering multiple motion
pHit = 0.6
pMiss = 0.2
# probability for inexact robot motion
pExact = 0.8 # probability for the correct move
pOvershoot = 0.1 # probability for overshooting the goal 
pUndershoot = 0.1 # probability for undershooting the goal

def sense(sense, prior):
    '''This function can calculate posterior probability given the sense and prior'''
    posterior = [0.0 for _ in range(len(prior))]
#     chaning prior according to the factor
    for i in range(len(prior)):
        if sense==world[i]:
            posterior[i] = prior[i] * pHit
        else:
            posterior[i] = prior[i] * pMiss
#     normalizing the posterior to make it a valid probability distribution
    posterior = [p/sum(posterior) for p in posterior]
    return posterior

def convolution(p, U):
    '''This function takes the posterior probability and steps to move in left or right.
    It returns new prior distribution. U=1 means move right by one cell, U=-1 means move left 
    by one cell. Assuming the world to be cyclic'''
    q = []
    for i in range(len(p)):
        s = pExact * p[(i-U) % len(p)] # calculating probability of correct motion
        s = s + pOvershoot * p[(i-U-1) % len(p)] # probability of overshoot motion
        s = s + pUndershoot * p[(i-U+1) % len(p)] # probability of undershoot motion
        q.append(s)
    return q

for i,s in enumerate(measurements):
    # step 2
    p = sense(s,p)
    # step 3
    p = convolution(p, motions[i])
print(p)

[0.21157894736842103, 0.1515789473684211, 0.08105263157894739, 0.16842105263157897, 0.3873684210526316]


So from the above output we can see that the probability distribution works in same way as previous but this time we consider the inexact robot motion. So this convolution function or move function is more accurate considering the real life scenario. 

### understand sense and move from probability theorem

If you look closely to the sense function it will lead us to the Bayesian rule. Lets represent the prior belief as X and the measurement as Z then the sense function is calculating the probability after having the measurement Z. In mathmatically we can say it like p(X|Z). We all know that according to the bayes theorem, 

    p(X|Z) = (p(Z|X) * P(X)) / p(Z)
    
Here p(Z|X) is the probability of having a measurement. In our case it is defined by pHit and pMiss. p(X) is the prior belief. p(Z) is just a normalized term. Since our final output is a posterior distribution we can replace p(Z) with just the normalization term. Thats the beauty of Bayes rule. So, 

    p(Z) = sum of p(Z|X) * P(X) for all the cells. 
    
Now the move or convolution function can be relate to something called total probability theory. The way we computed one cell probability after one move, was looking at all the grid cells from which it could have come from one time step earlier, we looked at the prior probability of those grid cells at previous time step and we multiply it with a probability that our motion command would carry us from those cells to this current cell. In probability term people write this like the follows-

    p(A) = sum over all B cells ( p(A|B) p(B) ) , here p(B) is the prior at previous time step , p(A|B) is probability of transition to this cell from B cells. 
    
Here A is the current cell index and B is the all possible previous cell's prior probabilities. This theorem is known as Theorem of total probability.  